# Table of Contents
 <p><div class="lev1 toc-item"><a href="#爬取单页内容" data-toc-modified-id="爬取单页内容-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>爬取单页内容</a></div><div class="lev1 toc-item"><a href="#读取单页内容" data-toc-modified-id="读取单页内容-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>读取单页内容</a></div><div class="lev1 toc-item"><a href="#爬取多页内容" data-toc-modified-id="爬取多页内容-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>爬取多页内容</a></div><div class="lev2 toc-item"><a href="#首页-&gt;-财经频道-&gt;-要闻-&gt;-公司新闻" data-toc-modified-id="首页->-财经频道->-要闻->-公司新闻-31"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>首页 &gt; 财经频道 &gt; 要闻 &gt; 公司新闻</a></div><div class="lev2 toc-item"><a href="#首页-&gt;-财经频道-&gt;-要闻-&gt;-上市公司-&gt;-列表" data-toc-modified-id="首页->-财经频道->-要闻->-上市公司->-列表-32"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>首页 &gt; 财经频道 &gt; 要闻 &gt; 上市公司 &gt; 列表</a></div><div class="lev1 toc-item"><a href="#读取多页内容" data-toc-modified-id="读取多页内容-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>读取多页内容</a></div><div class="lev2 toc-item"><a href="#读取-3.1" data-toc-modified-id="读取-3.1-41"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>读取 3.1</a></div><div class="lev2 toc-item"><a href="#读取-3.2" data-toc-modified-id="读取-3.2-42"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>读取 3.2</a></div><div class="lev2 toc-item"><a href="#集合数据" data-toc-modified-id="集合数据-43"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>集合数据</a></div><div class="lev1 toc-item"><a href="#标注数据" data-toc-modified-id="标注数据-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>标注数据</a></div><div class="lev2 toc-item"><a href="#标注细节" data-toc-modified-id="标注细节-51"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>标注细节</a></div><div class="lev3 toc-item"><a href="#实体标注细节" data-toc-modified-id="实体标注细节-511"><span class="toc-item-num">5.1.1&nbsp;&nbsp;</span>实体标注细节</a></div><div class="lev3 toc-item"><a href="#关系标注细节" data-toc-modified-id="关系标注细节-512"><span class="toc-item-num">5.1.2&nbsp;&nbsp;</span>关系标注细节</a></div>

# 爬取单页内容

In [101]:
import requests
from requests.exceptions import RequestException
import re
import json

In [102]:
def get_one_page(url):
    try:
        response = requests.get(url).content.decode('utf-8')
        if requests.get(url).status_code == 200:
            return response
        return None
    except RequestException:
        return None

def write_to_file(content):
    with open('single_page.json', 'a', encoding='utf-8') as f:
        f.write(json.dumps(content, ensure_ascii=False) + '\n')
        f.close()
        
def parse_one_page(html):
    pattern = re.compile('<p class="title">.*?<a href=".*?">(.*?)</a>.*?<p class="info" title="(.*?)">.*?<p class="time">(.*?)</p>'
                         , re.S)
    items = re.findall(pattern, html)
    for item in items:
        yield {
            'title': item[0],
            'abstract': item[1],
            'time': item[2][:-6]
        }    

In [103]:
url = 'http://finance.eastmoney.com/news/cgsxw_23.html'
html = get_one_page(url)
single_page = parse_one_page(html)
for s in single_page:
    write_to_file(s)

# 读取单页内容

In [4]:
import json
import pickle

In [14]:
with open('single_page.json') as f:
    lines = f.readlines()
    
title = []
abstract = []
time = []
for line in lines:
    item = json.loads(line)
    title.append(item['title'].strip())
    abstract.append(item['abstract'].strip())
    time.append('2017年' + item['time'].strip()[:-6])
    
tokens = (title, abstract, time)

In [15]:
len(title), len(abstract), set(time)

(18, 18, {'2017年06月12日'})

In [18]:
with open('tokens.pkl', 'wb') as fp:
    pickle.dump(tokens, fp)

In [19]:
with open('tokens.pkl', 'rb') as fp:
    title, abstract, time = pickle.load(fp)

In [21]:
title[0]

'日播时尚业绩滑坡库存高 女装三年13起抽检不合格'

In [22]:
abstract[0]

'中国经济网记者就上述问题向公司证券事务部发去采访函，截至发稿时未收到回复。中孚信息是专注于信息安全领域的高新技术企业，主营业务为信息安全产品研发、生产、销售及信息安全服务。公司主要产品及服务包括信息安全保密产品、商用密码产品、信息安全服务。信息安全保密产品服务对象主要为党政机关、军工企业、中央企业、科研院所等，商用密码产品服务领域主要为金融、税务、工商等。信息安全服务为涉密信息系统集成服务。公司坚持自主创新，研发了主要面向党政机关、军工企业、中央企业、科研院所的信息安全保密产品，部分产品处于国内领先地位。'

In [23]:
time[0]

'2017年06月12日'

# 爬取多页内容

## 首页 > 财经频道 > 要闻 > 公司新闻

In [1]:
import requests
from multiprocessing import Pool
from requests.exceptions import RequestException
import re
import json
import time

In [2]:
def get_one_page(url):
    try:
        response = requests.get(url).content.decode('utf-8')
        if requests.get(url).status_code == 200:
            return response
        return None
    except RequestException:
        return None

def write_to_file(content):
    with open('all_pages.json', 'a', encoding='utf-8') as f:
        f.write(json.dumps(content, ensure_ascii=False) + '\n')
        f.close()
        
def parse_one_page(html):
    pattern = re.compile('<p class="title">.*?<a href=".*?">(.*?)</a>.*?<p class="info" title="(.*?)">.*?<p class="time">(.*?)</p>'
                         , re.S)
    items = re.findall(pattern, html)
    for item in items:
        yield {
            'title': item[0],
            'abstract': item[1],
            'time': item[2]
        }    

def all_pages(num):
    url = 'http://finance.eastmoney.com/news/cgsxw_' + str(num) + '.html'
    html = get_one_page(url)
    for item in parse_one_page(html):
        write_to_file(item)

In [3]:
pool = Pool()
pool.map(all_pages, [i+1 for i in range(25)])

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [4]:
#start = time.time()
#for i in range(23):
#    all_pages(i+1)
#end = time.time()
#print('Time-consuming: %.3fs' % (end - start))

## 首页 > 财经频道 > 要闻 > 上市公司 > 列表

In [5]:
import requests
from multiprocessing import Pool
from requests.exceptions import RequestException
import re
import json
import time

In [6]:
def get_one_page(url):
    try:
        response = requests.get(url).content.decode('utf-8')
        if requests.get(url).status_code == 200:
            return response
        return None
    except RequestException:
        return None

def write_to_file(content):
    with open('all_pages_2.json', 'a', encoding='utf-8') as f:
        f.write(json.dumps(content, ensure_ascii=False, sort_keys=True) + '\n')
        f.close()
        
def parse_one_page(html):
    pattern = re.compile('<p class="title">.*?<a href=".*?">(.*?)</a>.*?<p class="info" title="(.*?)">.*?<p class="time">(.*?)</p>'
                         , re.S)
    items = re.findall(pattern, html)
    for item in items:
        yield {
            'title': item[0],
            'abstract': item[1],
            'time': item[2]
        }    

def all_pages(num):
    url = 'http://finance.eastmoney.com/news/cssgs_' + str(num) + '.html'
    html = get_one_page(url)
    for item in parse_one_page(html):
        write_to_file(item)

In [7]:
pool = Pool()
pool.map(all_pages, [i+1 for i in range(25)])

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [8]:
#start = time.time()
#for i in range(9):
#    all_pages(i+1)
#end = time.time()
#print('Time-consuming: %.3fs' % (end - start))

# 读取多页内容

## 读取 3.1

In [9]:
import json
import pickle

In [10]:
with open('all_pages.json') as f:
    lines = f.readlines()
       
title = []
abstract = []
time = []
for line in list(set(lines)):
    item = json.loads(line)
    title.append(item['title'].strip())
    abstract.append(item['abstract'].strip())
    time.append('2017年' + item['time'].strip()[:-6])
    
tokens = (title, abstract, time)

In [11]:
#with open('all_tokens.pkl', 'wb') as fp:
#    pickle.dump(tokens, fp)

In [12]:
#with open('all_tokens.pkl', 'rb') as fp:
#    title, abstract, time = pickle.load(fp)

In [13]:
len(lines), len(title), len(set(abstract)), len(time)

(3780, 2954, 2931, 2954)

In [14]:
len(set(lines)), len(set(title)), len(set(abstract)), len(set(time))

(2954, 2948, 2931, 12)

In [15]:
set(time)

{'2017年08月07日',
 '2017年08月08日',
 '2017年08月09日',
 '2017年08月10日',
 '2017年08月11日',
 '2017年08月12日',
 '2017年08月13日',
 '2017年08月14日',
 '2017年08月15日',
 '2017年08月16日',
 '2017年08月17日',
 '2017年08月18日'}

In [16]:
title[0]

'中国水务8月8日耗资439.94万港元回购101.6万股'

In [17]:
abstract[0]

'智通财经APP讯，中国信贷(08207)公布，2017年上半年，公司营业额达19.82亿元(人民币，下同)，同比增加428%；其中，在线投资及科技驱动借贷服务收入达14.90亿元，同比增加13.15倍。此外，期内溢利达7.29亿元，同比增加472.7%；公司拥有人应占净利达5.67亿元，同比增加347.3%；每股基本盈利达2.6分，不派中期股息。2017年6月5日，互金协会“互联网金融登记披露服务平台”正式上线运营，公司核心在线投资平台“网信”旗下“网信普惠”成为第一批10家接入的平台之一。'

In [18]:
time[0]

'2017年08月08日'

## 读取 3.2

In [19]:
import json
import pickle

In [20]:
with open('all_pages_2.json') as f:
    lines = f.readlines()
       
title = []
abstract = []
time = []
for line in list(set(lines)):
    item = json.loads(line)
    title.append(item['title'].strip())
    abstract.append(item['abstract'].strip())
    time.append('2017年' + item['time'].strip()[:-6])
    
tokens = (title, abstract, time)

In [21]:
#with open('all_tokens_2.pkl', 'wb') as fp:
#    pickle.dump(tokens, fp)

In [22]:
#with open('all_tokens_2.pkl', 'rb') as fp:
#    title, abstract, time = pickle.load(fp)

In [23]:
len(lines), len(title), len(abstract), len(time)

(2794, 1143, 1143, 1143)

In [24]:
len(set(lines)), len(set(title)), len(set(abstract)), len(set(time))

(1143, 1141, 1110, 12)

In [25]:
set(time)

{'2017年08月07日',
 '2017年08月08日',
 '2017年08月09日',
 '2017年08月10日',
 '2017年08月11日',
 '2017年08月12日',
 '2017年08月13日',
 '2017年08月14日',
 '2017年08月15日',
 '2017年08月16日',
 '2017年08月17日',
 '2017年08月18日'}

In [26]:
title[0]

'产能过剩格局未改 正海磁材半年营收净利双降'

In [27]:
abstract[0]

'【产能过剩格局未改 正海磁材半年营收净利双降】正海磁材8月14日晚间披露2017年半年报。报告显示，公司上半年实现营业收入4.03亿元，同比下降44.30%；净利润为-1496.94万元，上年同期为7289万元。这样的业绩与近期两个月股价的大幅上涨有些不匹配。（证券时报）'

In [28]:
time[0]

'2017年08月15日'

## 集合数据

In [29]:
import json
import pickle

In [30]:
title = []
abstract = []

In [31]:
with open('all_pages.json') as f1:
    lines1 = f1.readlines()
       
with open('all_pages_2.json') as f2:
    lines2 = f2.readlines()
        
for line in list(set(lines1)):
    item = json.loads(line)
    title.append(item['title'].strip())
    abstract.append(item['abstract'].strip())

for line in list(set(lines2)):
    item = json.loads(line)
    title.append(item['title'].strip())
    abstract.append(item['abstract'].strip())
    
title = list(set(title))
abstract = list(set(abstract))

In [32]:
with open('title.pkl', 'wb') as fp:
    pickle.dump(title, fp)
with open('abstract.pkl', 'wb') as fp:
    pickle.dump(abstract, fp)

In [33]:
#with open('title.pkl', 'rb') as fp:
#    title = pickle.load(fp)
#with open('abstract.pkl', 'rb') as fp:
#    abstract = pickle.load(fp)

In [34]:
len(title), len(abstract)

(3104, 2966)

In [35]:
title[1]

'京基陈家荣增持美图3487万股，每股11.668元'

In [36]:
abstract[1]

'鼎和矿业(00705)公布，2017年8月14日，公司与建源投资签订一份除保密条款外没有法律约束力的股权收购意向书，据此，公司拟向建源投资收购肇庆聚光光伏电站及云浮聚光光伏电站的全部股权。在签订意向书后，公司将向庆聚光及云浮聚光(其中包括)资产及负债等进行尽职调查，并在满意有关结果后在签订书向书3个月内签订正式收购协议。公告显示，肇庆聚光，注册资本人民币2000万元，经营范围为从事光伏发电项目的开发、建设、维护、光伏设备及元器件制造及销售。'

# 标注数据

In [1]:
import json
import pickle

```python
with open('abstract_data/01.pkl', 'rb') as fp:
    abstract = pickle.load(fp)
with open('abstract_data/02.pkl', 'rb') as fp:
    abstract_2 = pickle.load(fp)
abstract.extend(abstract_2)
abstract = list(set(abstract))
len(abstract)
```

In [2]:
with open('abstract_data/10.pkl', 'rb') as fp:
    abstract = pickle.load(fp)
abstract = list(set(abstract))
len(abstract)

4759

In [3]:
def write_to_file(content):
    with open('train/train_data.json', 'a', encoding='utf-8') as f:
        f.write(json.dumps(content, ensure_ascii=False) + '\n')
        f.close()
        
def gen_data(data):
    for item in data:
        yield {
            'abstract': item,
            'eneity': ["", ""],
            'relation': ""
        }

In [4]:
for i in gen_data(abstract):
    write_to_file(i)

## 标注细节

### 实体标注细节

### 关系标注细节

- 股权转让
- 无相关关系
- #收到监管措施事先告知书（证监会）
- 收购
- 合作
- 损失
- 失去控制权
- 提供技术支持
- 认购股票
- 购入股权
- #存在多种关系：子公司，
- 

In [42]:
with open('train/train_data.json') as f:
    lines = f.readlines()
       
abstract = []
eneity = []

for line in lines:
    item = json.loads(line)
    abstract.append(item['abstract'])
    eneity.append(item['eneity'])

In [43]:
len(abstract)

279

In [41]:
len(set(abstract))

8215

In [50]:
abstract[0]

'【东富龙：向迪安诊断转让重庆产研院股权】东富龙(300171)6月1日晚公告，公司下属全资子公司东富龙医疗与迪安诊断(300244)签署股权转让协议，约定东富龙医疗以5035万元，向迪安诊断转让重庆精准医疗产业技术研究院有限公司(简称重庆产研院)100%股权。（证券时报网）'

In [51]:
len(eneity)

7466

In [52]:
eneity[0]

['东富龙', '迪安诊断', '东富龙医疗', '重庆产研院']